# Match filter experiments

In [9]:
import xarray as xr
import pandas as pd
import numpy as np
import scipy.signal as signal
import scipy
import xrsignal
import os
from scipy import io
from matplotlib import pyplot as plt
from numpy import matlib
import hvplot.xarray

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
account_key= os.environ['AZURE_KEY_ooidata']
storage_options={'account_name': 'ooidata', 'account_key': account_key}
ds = xr.open_zarr('abfs://kauai/kauai_transmissions_ooi.zarr', storage_options=storage_options)
ds = ds.assign_coords({'time':np.arange(0,3600*2 + 1/1000, 1/1000)})

### create 20 min transmitted signal

In [3]:
fs, Tx = io.wavfile.read('tx_signal.wav')
t = np.arange(0, 65472/fs, 1/fs)

Tx1000 = signal.resample(Tx, int(len(Tx)/2.4))
t1000 = np.arange(0, len(Tx1000)/1000, 1/1000)

t_20min = np.arange(0,20*60, 1/1000)
Tx_20min = matlib.repmat(Tx1000, 1, 44).flatten()[:20*60*1000]

In [5]:
Tx_20min_x = xr.DataArray(Tx_20min, dims=['time'])

In [7]:
R12 = xrsignal.correlate_chunk(ds['AXCC1'], Tx_20min_x, mode='same')

In [10]:
R12.hvplot.line(x='time', rasterize=True)

:DynamicMap   [transmission]
   :Image   [time,value]   (time_value Count)